In [ ]:

import numpy as np
import pandas as pd
import netCDF4 as nc
import geopandas as gpd
from netCDF4 import num2date
import datetime

from tqdm import tqdm

tqdm.pandas()
from shapely.geometry import Point

import matplotlib.pyplot as plt

import platform

from cartopy import crs as ccrs

if platform.system() == 'Darwin':
    plt.rcParams["font.family"] = 'Arial Unicode MS'
elif platform.system() == 'Windows':
    plt.rcParams["font.family"] = 'SimHei'
else:
    pass

In [ ]:
nc_data = nc.Dataset("cru_ts4.05.1901.2020.tmp.dat.nc")
raw_time_data = np.array(nc_data.variables['time'])
raw_tmp_data = np.array(nc_data.variables['tmp'])

In [ ]:
def cftime2datetime(cftime, units, format='%Y-%m-%d %H:%M:%S'):
    """
    将nc文件里面的时间格式 从cftime 转换到 datetime格式
    :param cftime:
    :param units:
    :param format:
    :return:
    """
    return datetime.datetime.strptime(num2date(times=cftime, units=units).strftime(format), format)


clean_time_data = pd.Series([cftime2datetime(i, units='days since 1900-1-1') for i in raw_time_data])
clean_time_data[:4]

In [ ]:
clean_time_data.dt.year == 2020

In [ ]:
raw_tmp_data[clean_time_data.dt.year == 2020, :, :].shape

In [ ]:
raw_tmp_data[(1901 <= clean_time_data.dt.year) & (clean_time_data.dt.year <= 2012), :, :].shape

In [ ]:
tmp_missing_value = nc_data.variables['tmp'].missing_value

test = raw_tmp_data[clean_time_data == '1901-01', :, :]
test[test == tmp_missing_value] = np.nan

test = np.nanmean(test, axis=0)
test.shape

In [ ]:
test = raw_tmp_data[('1901-01' <=clean_time_data) & (clean_time_data <= '1902-02') , :, :]
test.shape

In [ ]:
test[test == tmp_missing_value] = np.nan

test = np.nanmean(test, axis=0)
test.shape

In [ ]:
clean_time_data